# nbforms Demo Notebook

The nbforms package allows you to have interactive questions in Jupyter Notebooks that is designed to allow immediate usage of collected data. It requires you to have deployed an [`nbforms-server`](https://github.com/chrispyles/nbforms-server); the demo server is located at https://nbforms-demo-server.herokuapp.com.

### Setup

Before using nbforms, you must provide a config file. The default location of the config file is `./nbforms_config.py`; the cell below loads the config file for this notebook for your perusal.

In [ ]:
with open("./nbforms_config.py") as f:
    print(f.read())

### Usage

To use nbforms, create a `Notebook` instance. This will load the config file and ask the use to authenticate with the server, generating an API token for this notebook session.

**If a user has never authenticated before, the cell below will create their user, which they can then reuse on the server. There is no need to go to another page to create a user.** The rules for logins are:
* If there is no user with the username provided, a new user is created with the provided password. Then, a new API key is generated and returned to the `Notebook` instance.
* If there is a user with the username provided, the password provided is compared against the user's password. If this passes, a new API key is generated and returned to the `Notebook` instance.
* If the passwords do not match, then the cell will throw an `AssertionError`.

In [ ]:
import nbforms
forms = nbforms.Notebook()

To ask a user to respond to a question, call `Notebook.ask`  with the question _identifier_ as its argument. The widget generated will have a "Submit" button which will send a POST request to your nbforms server that will record the user's response.

nbforms allows multiple choice (with one or many selections), text, and paragraph responses. An example of each is given below.

In [ ]:
forms.ask("q1")

In [ ]:
forms.ask("q2")

In [ ]:
forms.ask("q3")

In [ ]:
forms.ask("q4")

In [ ]:
forms.ask("q5")

#### Extracting Data from the Server

When retrieving the data from the server, nbforms allows you to collect the data into a datascience `Table` or a pandas `DataFrame`. To get the data from the server, use `Notebook.to_table` or `Notebook.to_df` and provide the question identifiers you would like to select. The optional `user_hashes` tells the server whether or not to include an randomly generated hash of each username in the CSV. *In order to protect user data, you cannot retrieve the usernames of users in the CSV.*

In [ ]:
forms.to_table("q1", "q2", "q3", "q4", "q5")

In [ ]:
forms.to_df("q1", "q4", "q5", user_hashes=True)